In [1]:
# Check GPU
!nvidia-smi

# Clone repo chính thức của StyleGAN2-ADA-PyTorch
!git clone --depth=1 https://github.com/NVlabs/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch

# Cài đặt các dependency cần thiết
!pip install ninja opensimplex==0.3 protobuf==3.20.3
!pip install imageio-ffmpeg==0.4.3

# Kiểm tra môi trường torch và CUDA
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))


Tue Oct 14 06:43:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


In [ ]:
import torch
import legacy
import dnnlib
import numpy as np
from PIL import Image
import pandas as pd
import os

# === Cấu hình ===
network_pkl = "/content/drive/MyDrive/Work_space/project_facial_expression/stylegan2_fer2013/outputs/00002-disgust-auto1-batch8-noaug-resumecustom/network-snapshot-000060.pkl"
outdir = "/content/drive/MyDrive/Work_space/project_facial_expression/stylegan2_fer2013/generate/generated_images"
num_images = 1300
truncation_psi = 1.0
device = torch.device('cuda')
output_csv = os.path.join(outdir, "image_vectors.csv")

# Create the output directory if it doesn't exist
os.makedirs(outdir, exist_ok=True)

# === Load model ===
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # Generator EMA

# === Sinh ảnh và lưu vector vào CSV ===
image_vectors = []
for seed in range(num_images):
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    img = G(z, None, truncation_psi=truncation_psi, noise_mode='const')
    img = (img.clamp(-1, 1) + 1) * (255 / 2)
    img = img.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)[0]

    # Flatten the image data into a vector
    img_vector = img.flatten()
    image_vectors.append(img_vector)

# Convert the list of vectors to a pandas DataFrame and save to CSV
df = pd.DataFrame(image_vectors)
df.to_csv(output_csv, index=False)


print(f"✅ Đã sinh {num_images} ảnh và lưu vector tại {output_csv}")

In [4]:
import torch
import legacy
import dnnlib
import numpy as np
from PIL import Image
import pandas as pd
import os
import csv

# === Cấu hình ===
network_pkl = "/content/drive/MyDrive/Work_space/project_facial_expression/stylegan2_fer2013/outputs/00002-disgust-auto1-batch8-noaug-resumecustom/network-snapshot-000060.pkl"
outdir = "/content/drive/MyDrive/Work_space/project_facial_expression/stylegan2_fer2013/generate/generated_images"
num_images = 1300
truncation_psi = 1.0
device = torch.device('cuda')
output_csv = os.path.join(outdir, "image_vectors.csv")

# Tạo thư mục nếu chưa có
os.makedirs(outdir, exist_ok=True)

# === Load model ===
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

# === Ghi CSV dần dần ===
with open(output_csv, mode='w', newline='') as f:
    writer = csv.writer(f)
    header_written = False

    for seed in range(num_images):
        # Sinh z và ảnh
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
        img = G(z, None, truncation_psi=truncation_psi, noise_mode='const')
        img = (img.clamp(-1, 1) + 1) * (255 / 2)
        img = img.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)[0]

        # Flatten
        img_vector = img.flatten()

        # Ghi header (cột 0...N)
        if not header_written:
            writer.writerow([f"pixel_{i}" for i in range(len(img_vector))])
            header_written = True

        # Ghi dữ liệu
        writer.writerow(img_vector.tolist())

        # Dọn dẹp để tránh chiếm RAM GPU
        del img, z
        torch.cuda.empty_cache()

        if (seed + 1) % 100 == 0:
            print(f"✅ Đã sinh {seed + 1}/{num_images} ảnh...")

print(f"\n🎉 Hoàn tất: {num_images} ảnh và lưu vector tại {output_csv}")


Setting up PyTorch plugin "bias_act_plugin"... 

W1014 06:48:27.004000 4624 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1014 06:48:27.004000 4624 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

W1014 06:48:27.748000 4624 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W1014 06:48:27.748000 4624 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


Done.
✅ Đã sinh 100/1300 ảnh...
✅ Đã sinh 200/1300 ảnh...
✅ Đã sinh 300/1300 ảnh...
✅ Đã sinh 400/1300 ảnh...
✅ Đã sinh 500/1300 ảnh...
✅ Đã sinh 600/1300 ảnh...
✅ Đã sinh 700/1300 ảnh...
✅ Đã sinh 800/1300 ảnh...
✅ Đã sinh 900/1300 ảnh...
✅ Đã sinh 1000/1300 ảnh...
✅ Đã sinh 1100/1300 ảnh...
✅ Đã sinh 1200/1300 ảnh...
✅ Đã sinh 1300/1300 ảnh...

🎉 Hoàn tất: 1300 ảnh và lưu vector tại /content/drive/MyDrive/Work_space/project_facial_expression/stylegan2_fer2013/generate/generated_images/image_vectors.csv


In [5]:
import torch
import legacy
import dnnlib
import numpy as np
from PIL import Image
# === Load model ===
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # Generator EMA
for seed in range(num_images):
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    img = G(z, None, truncation_psi=truncation_psi, noise_mode='const')
    img = (img.clamp(-1, 1) + 1) * (255 / 2)
    img = img.permute(0, 2, 3, 1).cpu().numpy().astype(np.uint8)[0]
    Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.png")

/tmp/ipython-input-2790228130.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.png")
/tmp/ipython-input-2790228130.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.png")
/tmp/ipython-input-2790228130.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.png")
/tmp/ipython-input-2790228130.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.png")
/tmp/ipython-input-2790228130.py:14: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img, 'RGB').save(f"{outdir}/img_{seed:03d}.p